# Error Analysis GPL

In [2]:
import pandas as pd
import json

In [55]:
with open('./data/gpl_zeroshot_retrieve_results.json') as f:
    d_retrieval_results = json.load(f)

d_retrieval_map = {}
for qid, value in d_retrieval_results.items():
    d_retrieval_map[qid] = list(value.keys())
df_retrieval = pd.DataFrame(list(d_retrieval_map.items()), columns=['qid', 'article_ids'])
df_test_golden = pd.read_csv('./data/questions_fr_test.csv')

def verify_recall(row):
    golden_articles = df_test_golden[df_test_golden["id"] == int(row["qid"])]["article_ids"].values.tolist()
    return any(i in golden_articles for i in row["article_ids"])

df_retrieval["is_retrieved"] = df_retrieval.apply(verify_recall, axis=1)
df_retrieval["qid"] = df_retrieval["qid"].astype('int')
df_retrieval_all = df_retrieval.merge(df_test_golden[["id", "category","question", "article_ids"]],left_on="qid", right_on="id", how='inner')
df_retrieval_all

,qid,article_ids_x,is_retrieved,id,category,question,article_ids_y
0,499,"[14912, 19098, 5383, 22550, 1746, 1282, 1017, ...",False,499,Famille,Je vis en union libre. Quels sont les droits d...,"944,945,946,947,948,949,950,951,1093,1094,1095..."
1,411,"[15764, 1111, 1878, 1198, 1966, 1701, 15889, 5...",False,411,Famille,J'ai signé une déclaration de cohabitation lég...,"2329,1676"
2,691,"[4391, 4374, 5962, 5421, 9870, 4846, 16441, 11...",False,691,Justice,Ai-je droit à l'aide juridique (ex pro deo) si...,"4478,4479,4480,4481,4482,4483,4484,4485,4486,4..."
3,632,"[14591, 15702, 15812, 2205, 4823, 15152, 6047,...",False,632,Justice,Puis-je refuser cette transaction pénale ?,13520
4,245,"[5527, 1356, 10195, 1423, 1300, 5408, 11542, 1...",False,245,Famille,Les père et mère de l'enfant résident séparéme...,"1274,1275,1276,1277,1278,1279,1280,1281,1282,1..."
...,...,...,...,...,...,...,...
217,471,"[5650, 3704, 1741, 5494, 1908, 1076, 22100, 12...",True,471,Famille,Je suis marié(e). Le propriétaire a envoyé son...,1071
218,116,"[15880, 15922, 1334, 2718, 14110, 15821, 20059...",True,116,Argent,Le règlement collectif de dettes suspend-il le...,5961
219,173,"[1766, 19116, 1679, 1687, 1821, 2196, 6556, 10...",False,173,Argent,Puis-je déshériter mes enfants ?,1818
220,1086,"[4816, 8252, 4566, 4417, 7107, 13604, 21358, 2...",False,1086,Travail,Je reçois les confidences en tant que professi...,6617


In [58]:
df_retrieval_failed_recall_100 = df_retrieval_all[df_retrieval_all["is_retrieved"] == False]
df_retrieval_failed_recall_100

,qid,article_ids_x,is_retrieved,id,category,question,article_ids_y
0,499,"[14912, 19098, 5383, 22550, 1746, 1282, 1017, ...",False,499,Famille,Je vis en union libre. Quels sont les droits d...,"944,945,946,947,948,949,950,951,1093,1094,1095..."
1,411,"[15764, 1111, 1878, 1198, 1966, 1701, 15889, 5...",False,411,Famille,J'ai signé une déclaration de cohabitation lég...,"2329,1676"
2,691,"[4391, 4374, 5962, 5421, 9870, 4846, 16441, 11...",False,691,Justice,Ai-je droit à l'aide juridique (ex pro deo) si...,"4478,4479,4480,4481,4482,4483,4484,4485,4486,4..."
3,632,"[14591, 15702, 15812, 2205, 4823, 15152, 6047,...",False,632,Justice,Puis-je refuser cette transaction pénale ?,13520
4,245,"[5527, 1356, 10195, 1423, 1300, 5408, 11542, 1...",False,245,Famille,Les père et mère de l'enfant résident séparéme...,"1274,1275,1276,1277,1278,1279,1280,1281,1282,1..."
...,...,...,...,...,...,...,...
213,771,"[585, 411, 12206, 14467, 611, 8983, 637, 7583,...",False,771,Logement,Je pense que mon logement est insalubre. Je m'...,"12016,12017"
215,934,"[15798, 241, 623, 758, 12938, 616, 12997, 355,...",False,934,Logement,Mon propriétaire peut-il fixer librement le pr...,861
216,819,"[9354, 12752, 5882, 3417, 1508, 12030, 5795, 3...",False,819,Logement,Je m'assure que le propriétaire a obtenu un pe...,"12031,12033"
219,173,"[1766, 19116, 1679, 1687, 1821, 2196, 6556, 10...",False,173,Argent,Puis-je déshériter mes enfants ?,1818
